## OpenAI Assistants API

The new Assistants API is a stateful evolution of Chat Completions API meant to simplify the creation of assistant-like experiences, and enable developer access to powerful tools like Code Interpreter and Retrieval.

![Alt text](assistants.png "Assistants")

## Chat Completions API vs Assistants API

The primitives of the Chat Completions API are Messages, on which you perform a Completion with a Model (gpt-3.5-turbo, gpt-4, etc). It is lightweight and powerful, but inherently stateless, which means you have to manage conversation state, tool definitions, retrieval documents, and code execution manually.

The primitives of the Assistants API are

1. Assistants, which encapsulate a base model, instructions, tools, and (context) documents,
2. Threads, which represent the state of a conversation, and
3. Runs, which power the execution of an Assistant on a Thread, including textual responses and multi-step tool use.


# Assistants API - Code Interpreter 

https://cookbook.openai.com/examples/assistants_api_overview_python

https://platform.openai.com/docs/assistants/overview

Watch:

https://www.youtube.com/watch?v=5rcjGjgJNQc

In [16]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [17]:
client : OpenAI = OpenAI()

### Code Interpreter

Code Interpreter allows the Assistants API to write and run Python code in a sandboxed execution environment. This tool can process files with diverse data and formatting, and generate files with data and images of graphs. Code Interpreter allows your Assistant to run code iteratively to solve challenging code and math problems. When your Assistant writes code that fails to run, it can iterate on this code by attempting to run different code until the code execution succeeds.

https://platform.openai.com/docs/assistants/tools/code-interpreter 

### Step 1: Create an Assistant

In [18]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)


### Step 2: Create a Thread

In [19]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()


### Step 3: Add a Message to a Thread

In [20]:


message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


### Step 4: Run the Assistant

In [21]:
from openai.types.beta.threads.run import Run

# run: Run = client.beta.threads.runs.create(
#   thread_id=thread.id,
#   assistant_id=assistant.id,
#   instructions="Please address the user as Jane Doe. The user has a premium account."
# )

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)


### Step 5: Check the Run status

In [22]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)


Run(id='run_QwxU5M0VriWVR9jytqKeogAO', assistant_id='asst_EibyXR9LIjZqBCM7uGvqBbvN', cancelled_at=None, completed_at=1718204538, created_at=1718204528, expires_at=None, failed_at=None, incomplete_details=None, instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, response_format='auto', started_at=1718204529, status='completed', thread_id='thread_W5vD281I1qwnmJly9LcrbfgT', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=84, prompt_tokens=345, total_tokens=429), temperature=1.0, top_p=1.0, tool_resources={}, parallel_tool_calls=True)


### Step 6: Display the Assistant's Response

In [23]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)


user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: The solution to the equation 3x + 11 = 14 is x = 1.


## Streaming Assistant

In [24]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 

# Add Message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > We have already solved the equation and found that the solution is x = 1.